In [ ]:
!pip install geopandas
!pip install backoff
!pip install statsmodels
!pip install seaborn
!pip install scipy
!pip install mapclassify
!pip install unidecode
!pip install rasterio
!pip install seaborn
!pip install mpl_scatter_density
!pip install rasterstats
!pip install xmltodict
!pip install pandas

In [2]:
import os
import subprocess
import geopandas
import pandas as pd
import json
import datetime
import backoff
import xmltodict
import fileinput
import xml.etree.ElementTree as ET
from urllib.parse import urlparse
import fileinput
import glob
import matplotlib.pyplot as plt
# import numpy as np
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings( "ignore", module = "matplotlib\..*" )
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import scipy.stats
import scipy.stats as stats
from statsmodels import graphics
import shutil
from mpl_toolkits.axes_grid1 import make_axes_locatable
from unidecode import unidecode
import rasterio
from rasterio.plot import show
import seaborn
# import mpl_scatter_density # adds projection='scatter_density'
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors
from textwrap import wrap
import matplotlib.patches as mpatches
import pylab as py
import rasterstats
from rasterstats import zonal_stats
import scipy
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pandas as pd
import subprocess
import glob
import shutil
import time

from maap.maap import MAAP
# maap=MAAP(maap_host="api.ops.maap-project.org") 
maap=MAAP() 

def BUILD_TABLE_JOBSTATUS(submit_results_df, status_col = 'status'):
    import xmltodict
    
    # If jobs failed to submit, then they have a NaN for jobid, which makes the merge (join) fail
    submit_results_df = submit_results_df.fillna('')
    
    job_status_df = pd.concat([pd.DataFrame({'job_id': [job_id], 'status':[maap.getJobStatus(job_id)]}) for job_id in submit_results_df.job_id.to_list()])
    job_status_df = submit_results_df.merge(job_status_df, how='left', left_on='job_id',  right_on='job_id')
    
    print(f'Count total jobs:\t{len(job_status_df)}')
    print(f"Count pending jobs:\t{job_status_df[job_status_df[status_col] =='Accepted'].shape[0]}")
    print(f"Count running jobs:\t{job_status_df[job_status_df[status_col] =='Running'].shape[0]}")
    
    NUM_FAILS = job_status_df[job_status_df[status_col] =='Failed'].shape[0]
    NUM_SUCCEEDS = job_status_df[job_status_df[status_col] =='Succeeded'].shape[0]
    print(f"Count succeeded jobs:\t{NUM_SUCCEEDS}")
    print(f"Count failed jobs:\t{NUM_FAILS}")
    if NUM_FAILS > 0:
        print(f"% of failed jobs:\t{round(NUM_FAILS / ( NUM_FAILS + NUM_SUCCEEDS ), 4) * 100}\n")
    else:
        print(f"% of failed jobs:\tNothing has failed...yet\n")
    
    return job_status_df

### DPS Jobs for all TILES

#### COMMIT ALL CHANGES and REGISTER ALGORITHM FROM TERMINAL

In [3]:
# cd /projects/ADE_biomass_harmonization/NASA_CMS
# git add NASA_CMS_2023/Mexico/
# git commit -m 'Running batch predictions'
# git tag -f GMBpredictions_v1

# git push origin -f GMBpredictions_v1
# git push
# git push origin -f GMBpredictions_v1

In [5]:
########### REGISTER ALGORITHM ON NEW ADE ##############
maap.register_algorithm_from_yaml_file("/projects/ADE_biomass_harmonization/NASA_CMS/NASA_CMS_2023/Mexico/DPS_algorithm/run_GMBpredictions_MEX.yaml").text

'{"code": 200, "message": {"id": "e6f4105d0070522f15ceb42cef9c6e807756b82e", "short_id": "e6f4105d", "created_at": "2024-05-23T00:17:06.000+00:00", "parent_ids": ["bf409eb4c8b285a8a1e1321a10f2ec55abdeea9f"], "title": "Registering algorithm: run_GMBpredictions_MEX", "message": "Registering algorithm: run_GMBpredictions_MEX", "author_name": "root", "author_email": "root@88830aff274b", "authored_date": "2024-05-23T00:17:06.000+00:00", "committer_name": "root", "committer_email": "root@88830aff274b", "committed_date": "2024-05-23T00:17:06.000+00:00", "trailers": {}, "web_url": "https://repo.maap-project.org/root/register-job-hysds-v4/-/commit/e6f4105d0070522f15ceb42cef9c6e807756b82e", "stats": {"additions": 7, "deletions": 7, "total": 14}, "status": "running", "project_id": 3, "last_pipeline": {"id": 13771, "iid": 1144, "project_id": 3, "sha": "e6f4105d0070522f15ceb42cef9c6e807756b82e", "ref": "main", "status": "running", "source": "push", "created_at": "2024-05-23T00:17:07.597Z", "updated

#### RUN ALGORITHM FROM TERMINAL / SUBMIT JOBS TO DPS FOR ALL TILES ####

In [6]:
Desired_output_folder = '/projects/my-public-bucket/Data/NASA_CMS_2023/MEXICO/11_Merge_PSAyMFCcCyPM_Dummies_Rdo_PREDICTIONS/'
if not os.path.exists(Desired_output_folder):
    os.mkdir(Desired_output_folder)

Projects_folder  = '/projects/my-public-bucket/Data/NASA_CMS_2023/MEXICO/11_Merge_PSAyMFCcCyPM_Dummies_Rdo_SPLIT/'
TILES = [os.path.abspath(os.path.join(Projects_folder, p)) for p in os.listdir(Projects_folder) if p.endswith('.gpkg')]

i = 0
count = -1
submit_results_df_list = []
len_input_list = len(TILES)
list_jobIDs = [""]*len(TILES)

for TILE in TILES: 
    if not os.path.exists(os.path.join(Desired_output_folder,os.path.basename(TILE.split('.')[0]) + "_GMB_predictions.csv")): 
        print("submitting " + os.path.basename(TILE))
        count +=1
        submit_result = maap.submitJob(identifier="run_GMBpredictions_MEX", 
                       algo_id="run_GMBpredictions_MEX",
                       version="GMBpredictions_v1",
                       username="nehajo88",
                       queue="maap-dps-worker-16gb",                                     
                       mexico="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MEXICO/ecort08gw_DESECON1_DISS.gpkg",
                       ccirast="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MEXICO/cci_mexico_6933.tif",
                       heirast="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MEXICO/GLAD_FH_mexico_UINT16_C_6399.tif",
                       data="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MEXICO/NFI_CCI_GEDIheights.csv",
                       INLA_model_fit_v2="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MEXICO/INLA_model_fit_v2.RData",
                       MGN2020_INEGI_Urban_Mex_mask_DISS="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MEXICO/MGN2020_INEGI_Urban_Mex_mask_DISS.gpkg",
                       All_Products_Comp_Over30_Binary_6933="https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MEXICO/All_Products_Comp_Over30_Binary_6933.tif",
                       Projects=os.path.join("https://maap-ops-workspace.s3.amazonaws.com/shared/nehajo88/Data/NASA_CMS_2023/MEXICO/11_Merge_PSAyMFCcCyPM_Dummies_Rdo_SPLIT",os.path.basename(TILE)),
                       output_csv=os.path.basename(TILE.split('.')[0]) + "_GMB_predictions.csv")
        job_id = str(submit_result).split(': ')[1].split(',')[0].replace("'","")
        print(job_id)
        time.sleep(1)

submitting PROJECTS_2.gpkg
e66c54af-0ae3-4539-b6e8-1266ee4e9be2
submitting PROJECTS_3.gpkg
99f1fe94-2368-4b4d-b642-aaf168449533


##### TEST ON TERMINAL

```
Rscript /projects/NASA_CMS/NASA_CMS_2023/Mexico/DPS_algorithm/GMB_Mexico.R \
PROJECTS_1_GMB_predictions.csv \
/projects/shared-buckets/alexdevseed/NASA_CMS/PROJECTS_test.gpkg \
/projects/shared-buckets/nehajo88/Data/NASA_CMS_2023/MEXICO/ecort08gw_DESECON1_DISS.gpkg \
/projects/shared-buckets/nehajo88/Data/NASA_CMS_2023/MEXICO/cci_mexico_6933.tif \
/projects/shared-buckets/nehajo88/Data/NASA_CMS_2023/MEXICO/GLAD_FH_mexico_UINT16_C_6399.tif \
/projects/shared-buckets/nehajo88/Data/NASA_CMS_2023/MEXICO/NFI_CCI_GEDIheights.csv \
/projects/shared-buckets/nehajo88/Data/NASA_CMS_2023/MEXICO/INLA_model_fit_v2.RData \
/projects/shared-buckets/nehajo88/Data/NASA_CMS_2023/MEXICO/MGN2020_INEGI_Urban_Mex_mask_DISS.gpkg \
/projects/shared-buckets/nehajo88/Data/NASA_CMS_2023/MEXICO/All_Products_Comp_Over30_Binary_6933.tif
```


